# Scrape Online Travel Agent (OTA) Property Prices

@Nileka add a description here as to what this notebook offers

### Execute the cell below once to turn off debugging

In [1]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore')     #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True

## Initialize the OTA scrape class

In [2]:
import os
import sys
from datetime import datetime, date, timedelta

ROOT_DIR = "/home/nuwan/workspace/rezgate/wrangler"
UTILS_DIR = "/home/nuwan/workspace/rezgate/utils/"
MODULE_DIR = os.path.join(ROOT_DIR, 'modules/ota/')
DATA_DIR = os.path.join(ROOT_DIR, 'data/hospitality/bookings/scraper')
sys.path.insert(1,MODULE_DIR)
import otaUtils as otau
import propertyScrapers as ps
sys.path.insert(1,UTILS_DIR)
import sparkwls as spark
import nlp

''' restart initiate classes '''
if debug:
    import importlib
    ps = importlib.reload(ps)
    otau = importlib.reload(otau)
    spark = importlib.reload(spark)
    nlp = importlib.reload(nlp)
    
''' optional - if not specified class will use the default values '''
kwargs = {
    "ROOT_DIR":ROOT_DIR,   # absolute path to the wrangler dir
    "UTILS_DIR":UTILS_DIR, # abslute path to the generic utils
    "MODULE_DIR":MODULE_DIR,   # absolute path to the ota module
    "DATA_DIR":DATA_DIR, # absolute path to the scraper data dir
}
clsScraper = ps.PropertyScraper(desc='scrape hotel prices data from OTAs', **kwargs)
clsSparkWL = spark.SparkWorkLoads(desc="ota prices", **kwargs)
print("/nClass initialization and load complete!")

All Utils software packages loaded successfully!
All packages in SparkWorkLoads loaded successfully!
All PropertyScraper software packages loaded successfully!
All packages in SparkWorkLoads loaded successfully!
All PropertyScraper software packages loaded successfully!
All Utils software packages loaded successfully!
All packages in SparkWorkLoads loaded successfully!
All packages in SparkWorkLoads loaded successfully!
Initialing Utils class for otaUtils with instance Utilities class for property data scraping
Logging Utils info, warnings, and error to /home/nuwan/workspace/rezgate/wrangler/logs/modules/ota/app.log
Initialing NatLanWorkLoads class for nlp with instance classifying ota room types
Logging NatLanWorkLoads info, warnings, and error to /home/nuwan/workspace/rezgate/utils/logs/app.log
Initialing PropertyScraper class for propertyScraper with instance scrape hotel prices data from OTAs
Logging PropertyScraper info, warnings, and error to /home/nuwan/workspace/rezgate/wrangle

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Initialing SparkWorkLoads class for sparkWorkLoads with instance ota prices
Logging SparkWorkLoads info, warnings, and error to /home/nuwan/workspace/rezgate/utils/logs/app.log
/nClass initialization and load complete!


## Initialize the set of OTA URL for Scraping

In [142]:
#DATA_PATH = os.path.join(ROOT_DIR, 'data/hospitality/scraper/')
#path = DATA_PATH
file = "otaInputURLs.json"
start_date = date.today()
end_date = start_date + timedelta(days=1)

try:
    if start_date < date.today():
        raise ValueError("Start date must be greater than today: %s" % str(date.today()))
    if end_date <= start_date:
        raise ValueError("End date %s is invalid. It must be greater than Start Date: %s" % (str(end_date),str(start_date)))
    kwargs = {"pageOffset":10,
              "pageUpperLimit":550,
              "startDate": start_date,
              "endDate" : end_date,
             }

    _otaURLfilePath, _ota_url_parameterized_list  = clsScraper.build_scrape_url_list(
                                                    file_name=file,  # mandatory to give the inputs json file
                                                    dir_path=None,   # optional to be used iff required
                                                    **kwargs
                                                    )
    if _otaURLfilePath:
        print("Data in: %s" % _otaURLfilePath)
    if len(_ota_url_parameterized_list)>0:
        print("Completed parameterizing urls with %d instances." 
              % (len(_ota_url_parameterized_list)))

except Exception as err:
    _s_fn_id = "Class <WebScraper> Function <read_folder_csv_to_df>"
    print("[Error]"+_s_fn_id, err)
    print(traceback.format_exc())


Loaded 1 properties to begin scraping OTA data.
Processing booking.com ...
Data in: /home/nuwan/workspace/rezgate/wrangler/data/tmp/PropertyScraper-build-scrape-url-list.csv
Completed parameterizing urls with 784 instances.


## Prepare folder & file structure

In [143]:
from datetime import datetime, timezone

dirPath = None
_search_dt = datetime.now()
_search_dt = _search_dt + (datetime.min - _search_dt) % timedelta(minutes=15)
''' include the timezone '''
_search_dt = (_search_dt.replace(tzinfo=timezone.utc)).isoformat()

kwargs = {
#    'SEARCH_DATETIME': _search_dt,
    'STORAGE_METHOD': "local",   # values can be "local" or "AWS_S3"
}

#_current_search_data_dir = clsScraper.get_search_data_dir_path(dirPath, **kwargs)
_current_search_data_store_dir = clsScraper.make_storage_dir(**kwargs)
print("Extracting data into %s for search datetime: %s" 
      % (_current_search_data_store_dir,str(_search_dt)))

Extracting data into /home/nuwan/workspace/rezgate/wrangler/data/hospitality/bookings/scraper/rates/2022-9-23-20-30/ for search datetime: 2022-09-23T20:15:00+00:00


## Scrape data into CSVs
___TODO:___ hault if internet connection times out; might be possible with airflow.

In [146]:
import pandas as pd

if _otaURLfilePath:
    urlDF = pd.read_csv(_otaURLfilePath, sep=",")
    _otaURLParamDictList = urlDF.to_dict('records')

''' TODO change the function parameter names to camel format '''
_l_saved_files = clsScraper.scrape_url_list(
    otaURLlist =_otaURLParamDictList,
#    otaURLfile = None,
    searchDT = _search_dt,
    data_store_dir =_current_search_data_store_dir)
print("Scraping completed and data saved in %s!" % _current_search_data_store_dir)

loading parameterized urls from list 784 records
[Error]function <_scrape_bookings_to_csv> No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=20079110&dest_type=city&checkin=2022-09-23&checkout=2022-09-24&group_adults=1&no_rooms=1&group_children=0&selected_currency=USD&offset=150
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/wrangler/modules/ota/propertyScrapers.py", line 459, in _scrape_bookings_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gC

[Error]function <_scrape_bookings_to_csv> No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=20079110&dest_type=city&checkin=2022-09-23&checkout=2022-09-24&group_adults=1&no_rooms=1&group_children=0&selected_currency=USD&offset=220
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/wrangler/modules/ota/propertyScrapers.py", line 459, in _scrape_bookings_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=3

[Error]function <_scrape_bookings_to_csv> No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=20079110&dest_type=city&checkin=2022-09-23&checkout=2022-09-24&group_adults=1&no_rooms=1&group_children=0&selected_currency=USD&offset=290
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/wrangler/modules/ota/propertyScrapers.py", line 459, in _scrape_bookings_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=3

[Error]function <_scrape_bookings_to_csv> No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=20079110&dest_type=city&checkin=2022-09-23&checkout=2022-09-24&group_adults=1&no_rooms=1&group_children=0&selected_currency=USD&offset=360
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/wrangler/modules/ota/propertyScrapers.py", line 459, in _scrape_bookings_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=3

[Error]function <_scrape_bookings_to_csv> No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=20079110&dest_type=city&checkin=2022-09-23&checkout=2022-09-24&group_adults=1&no_rooms=1&group_children=0&selected_currency=USD&offset=430
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/wrangler/modules/ota/propertyScrapers.py", line 459, in _scrape_bookings_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=3

...

[Error]function <_scrape_bookings_to_csv> No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=20021296&dest_type=city&checkin=2022-09-23&checkout=2022-09-24&group_adults=1&no_rooms=1&group_children=0&selected_currency=USD&offset=260
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/wrangler/modules/ota/propertyScrapers.py", line 459, in _scrape_bookings_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=3

[Error]function <_scrape_bookings_to_csv> No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=20021296&dest_type=city&checkin=2022-09-23&checkout=2022-09-24&group_adults=1&no_rooms=1&group_children=0&selected_currency=USD&offset=330
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/wrangler/modules/ota/propertyScrapers.py", line 459, in _scrape_bookings_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=3

[Error]function <_scrape_bookings_to_csv> No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=20021296&dest_type=city&checkin=2022-09-23&checkout=2022-09-24&group_adults=1&no_rooms=1&group_children=0&selected_currency=USD&offset=400
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/wrangler/modules/ota/propertyScrapers.py", line 459, in _scrape_bookings_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=3

[Error]function <_scrape_bookings_to_csv> No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=20021296&dest_type=city&checkin=2022-09-23&checkout=2022-09-24&group_adults=1&no_rooms=1&group_children=0&selected_currency=USD&offset=470
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/wrangler/modules/ota/propertyScrapers.py", line 459, in _scrape_bookings_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=3

[Error]function <_scrape_bookings_to_csv> No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=20021296&dest_type=city&checkin=2022-09-23&checkout=2022-09-24&group_adults=1&no_rooms=1&group_children=0&selected_currency=USD&offset=540
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/wrangler/modules/ota/propertyScrapers.py", line 459, in _scrape_bookings_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=3

## Use Spark to Load Data into Database

### [REMOVE ATER DEBUG] Declare sparkworkload class

In [80]:
import os
import sys
from datetime import datetime, date, timedelta

#ROOT_DIR = "/home/nuwan/workspace/rezgate/wrangler/"
#UTILS_PATH = os.path.join(ROOT_DIR, 'utils/')
sys.path.insert(1, UTILS_DIR)
import sparkwls as spark
import nlp

if debug:
    import importlib
    spark = importlib.reload(spark)
    ps = importlib.reload(ps)
    nlp = importlib.reload(nlp)
    
kwargs = {
    "ROOT_DIR":ROOT_DIR,
}
clsScraper = ps.PropertyScraper(desc='scrape hotel prices data from OTAs', **kwargs)
clsSparkWL = spark.SparkWorkLoads(desc="ota prices", **kwargs)
#_session = clsSparkWL.get_spark_session()

All packages in SparkWorkLoads loaded successfully!
All PropertyScraper software packages loaded successfully!
All packages in SparkWorkLoads loaded successfully!
Initialing Utils class for otaUtils with instance Utilities class for property data scraping
Logging Utils info, warnings, and error to /home/nuwan/workspace/rezgate/wrangler/logs/modules/ota/app.log
Initialing NatLanWorkLoads class for nlp with instance classifying ota room types
Logging NatLanWorkLoads info, warnings, and error to /home/nuwan/workspace/rezgate/utils/logs/app.log
Initialing SparkWorkLoads class for sparkWorkLoads with instance ota property price scraper
Logging SparkWorkLoads info, warnings, and error to /home/nuwan/workspace/rezgate/utils/logs/app.log
Initialing PropertyScraper class for propertyScraper with instance scrape hotel prices data from OTAs
Logging PropertyScraper info, warnings, and error to /home/nuwan/workspace/rezgate/wrangler/logs/modules/ota/app.log
Data path set to /home/nuwan/workspace/re

### Read CSV into spark dataframe

In [5]:
spark_kwargs = {"TO_PANDAS":True}
#dirPath = _current_search_data_dir
dirPath = "../../data/hospitality/bookings/scraper/rates/2022-9-24-20-0/"
_search_sdf = clsSparkWL.read_csv_to_sdf(filesPath=dirPath, **spark_kwargs)
#_search_sdf = _search_sdf.distinct()
print("Spark loaded %d rows" % _search_sdf.shape[0])

Spark loaded 1084 rows


## Transform the data
augment the dataframe with: 
1. exctract the room rate decimal numbers
1. matching city names to the codes 
1. categorizing the room types based on the taxonomy
1. setting the data types of the columns
using a transform function in the properties class

### Extract room rate decimal value

In [6]:
''' define room price column to extrac number'''
rate_col_name = "room_rate"
aug_col_name = "room_price"
''' extract the price value from room rate'''
_search_sdf = clsScraper.extract_room_rate(_search_sdf,rate_col_name,aug_col_name)

### Categorize room type by similarity mappring

In [26]:
_save_rcate_to = os.path.join(ROOT_DIR,'data/tmp/similarity_categorized_rooms.csv')
''' categorize the room types '''
emb_kwargs = {
    'LOWER':True,
    'NO_STOP_WORDS':False,
    'METRIC':"COSIN",
    'MAX_SCORES':2,
    'TOLERANCE':0.7,
    'ROOM_CATE_FNAME':"room_descriptions.csv",
}
print("wait a moment this may take a while categorizing %d room type" % (_search_sdf.shape[0]))
# _room_type_assign = clsScraper.merge_similar_room_cate(_search_sdf,emb_kwargs)
_categorized_room_df = clsScraper.merge_similar_room_cate(_search_sdf,emb_kwargs)
#_categorized_room_df.to_csv(_save_rcate_to)
# print("Merged data saved to %s" % (_save_rcate_to))
print("Room type categorization complete!")

wait a moment this may take a while categorizing 1084 room type


KeyboardInterrupt: 

### Assign location city names

In [48]:
# destDirPath = os.path.join(DATA_DIR, 'destinations/')
_aug_dest_df = clsScraper.assign_lx_name(data_df=_categorized_room_df,
#                                          dest_dir_name=None
                                        )
print("merged %d rows with destination name and type" % _aug_dest_df.shape[0])

merged 18912 rows with destination name and type


## Save cleaned SDF to Tmp File

In [59]:
''' convert dataframe from panas to spark '''
_tmp_fname = clsSparkWL.save_sdf_to_csv(_aug_dest_df)
# print("%d rows saved to %s" % (aug_search_sdf.count(), _tmp_fname))
print("transformed data saved to %s" % _tmp_fname)

22/10/04 10:30:42 WARN TaskSetManager: Stage 22 contains a task of very large size (1023 KiB). The maximum recommended task size is 1000 KiB.
22/10/04 10:30:43 WARN TaskSetManager: Stage 25 contains a task of very large size (1023 KiB). The maximum recommended task size is 1000 KiB.
22/10/04 10:30:43 WARN TaskSetManager: Stage 28 contains a task of very large size (1023 KiB). The maximum recommended task size is 1000 KiB.


22/10/04 10:30:44 WARN TaskSetManager: Stage 29 contains a task of very large size (1023 KiB). The maximum recommended task size is 1000 KiB.
transformed data saved to /home/nuwan/workspace/rezgate/utils/data/tmp/SparkWorkLoadssave_sdf_to.csv


## Read CSV from Tmp and Save to DB Table

In [82]:
''' Save dataframe to table '''
_s_tbl_name = "ota_property_prices"
_get_tmp_sdf = clsSparkWL.read_csv_to_sdf(filesPath=_tmp_fname)
_get_tmp_sdf.printSchema()
count, saved_df = clsScraper.save_to_db(data_df=_aug_dest_df,table_name = _tbl_name)
# count = clsSparkWL.insert_sdf_into_table(save_sdf=_get_tmp_sdf, dbTable=_s_tbl_name)
print("%d Data saved to %s" % (saved_df,_s_tbl_name))

18912 Data saved to ota_property_prices


## Read data from table

In [84]:
_s_tbl_name = "ota_property_prices"
data = clsSparkWL.get_data_from_table(dbTable=_s_tbl_name)
print("%d records loaded from %s" %(data.count(),_s_tbl_name))

data.select("*").distinct().where(col('created_dt') >= date.today(), col()).sort("search_dt").show(2, vertical=True)

Wait a moment, retrieving data ...
Loading complete!
63426 records loaded from ota_property_prices


-RECORD 0------------------------------
 uuid           | null                 
 ota_name       | booking.com          
 search_dt      | 2022-09-25 04:00:00  
 property_name  | Hampton Inn Manha... 
 checkin_date   | 2022-09-24 00:00:00  
 checkout_date  | null                 
 room_type      | Queen Room           
 room_cate      | studio apartment ... 
 similarity     | 0.7373034358024597   
 room_rate      | US$393               
 room_price     | 393.0                
 currency       | US$                  
 review_score   | 8.0                  
 destination_id | 20088325             
 dest_lx_name   | New York City        
 dest_lx_type   | city                 
 location_desc  | Chelsea, New York... 
 other_info     | Queen Room1 large... 
 data_source    | null                 
 data_owner     | null                 
 created_dt     | 2022-10-04 10:31:... 
 created_by     | postgres             
 created_proc   | Unknown              
 modified_dt    | null                 


In [150]:
''' DEPRECATED -- moved as a function in otaPropertyScraper class '''

''' Get destination id dictionary '''
destDirPath = os.path.join(DATA_PATH, 'destinations/')
destinations_sdf = clsSparkWL.read_csv_to_sdf(filesPath=destDirPath)
destinations_sdf = destinations_sdf.selectExpr("city as destination_name", \
                                                "destinationID as destination_id")
# destinations_sdf = destinations_sdf.withColumn("destination_name",col("destination_name").cast(StringType())) \
#                                 .withColumn("destination_id",col("destination_id").cast(StringType()))
if debug:
    destinations_sdf.printSchema()
    destinations_sdf.show(n=2, vertical=True, truncate=False)
    print("Destination dictionary loarded!")

''' Lookup & augment destination name '''
#aug_search_sdf = destinations_sdf.join(_search_sdf,on='destination_id',how='rightouter')
aug_search_sdf = _search_sdf.join(destinations_sdf,
                                  _search_sdf.destination_id == destinations_sdf.destination_id,
                                  how='leftouter').drop(_search_sdf.destination_id)

if debug:
    print("%d destination names augmented to dataframe!" % (aug_search_sdf.count()))
    aug_search_sdf.show(n=2, vertical=True, truncate=False)

root
 |-- destination_name: string (nullable = true)
 |-- destination_id: integer (nullable = true)

-RECORD 0-------------------------
 destination_name | Las Vegas     
 destination_id   | 20079110      
-RECORD 1-------------------------
 destination_name | New York City 
 destination_id   | 20088325      
only showing top 2 rows

Destination dictionary loarded!


2095 destination names augmented to dataframe!


-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 search_dt        | 2022-09-24 02:45:00                                                                                                                                                                                                                                     
 checkin_date     | 2022-09-23 00:00:00                                                                                                                                                                                                                                     
 property_name    | Extended Stay America Suites - Orlando - Lake Buena Vista                                                                                                                    

In [8]:
''' DEPRECATED - replaced with a function on otaPropertyScrpaer calss '''

from pyspark.sql.functions import substring,lit,col
from pyspark.sql.types import StringType,BooleanType,DateType,DecimalType,FloatType, IntegerType,LongType, ShortType, TimestampType

# _search_sdf=_search_sdf.withColumn("currency", lit("US$"))
# _search_sdf=_search_sdf.withColumn('room_rate', substring('room_rate', 4,10))

# ''' reset data types to match table '''
# _search_sdf = _search_sdf.withColumn("destination_id",col("destination_id").cast(StringType())) \
#     .withColumn("room_rate",col("room_rate").cast(FloatType()))
# #    .withColumn("search_datetime",col("search_datetime").cast(DateType()))

if debug:
    _search_sdf.printSchema()
    _search_sdf.show(n=2, vertical=True, truncate=False)
print("Split and Extraction complete!")

AttributeError: 'DataFrame' object has no attribute 'printSchema'

In [18]:
''' DEPRECATED -- merge moved to the function in otaPropScraper '''

_categorized_room_df = _search_sdf.merge(_room_type_assign, how='left', left_on=['room_type'], right_on=['room_type'])
print("Merged %d rows with categorized room type information" % _categorized_room_df.shape[0])

_categorized_room_df = new_df
#_categorized_room_df.to_csv(_save_rcate_to)
#_save_rcate_to=clsSparkWL.save_sdf_to_csv(_save_rcate_to)
print("Merged data saved to %s" % (_save_rcate_to))
print("Room type categorization complete!")

Merged data saved to None
Room type categorization complete!


In [9]:
''' DEPRECATED '''
_categorized_room_df = _search_sdf.merge(_room_type_assign, how='left', left_on=['room_type'], right_on=['room_type'])
_categorized_room_df.to_csv(os.path.join(ROOT_DIR,'data/tmp/similarity_scores.csv'))